In [1]:
import requests, bs4, time, csv, math
import pandas as pd

In [77]:
m_url = open('michelin_urls.csv')
m_url = csv.reader(m_url)
m_url = list(m_url)
m_url = m_url[1:]

In [4]:
def get_restaurant_data(d):

    r_dict = {}
    restaurant_name = 'Unknown' if d.select('h1.biz-page-title.embossed-text-white')[0].string is None else\
    d.select('h1.biz-page-title.embossed-text-white')[0].string.encode('ascii','ignore').strip()
    
    overall_rating = 'Unknown' if d.find_all('meta', {'itemprop': 'ratingValue'})[0].get('content') is None else\
    d.find_all('meta', {'itemprop': 'ratingValue'})[0].get('content')
    
    total_reviews = 'Unknown' if d.find_all('span', {'itemprop': 'reviewCount'})[0].string is None else\
    d.find_all('span', {'itemprop': 'reviewCount'})[0].string
    
    price_range = 'Unknown' if d.find_all('span', {'class': 'business-attribute price-range'})[0].string is None else\
    d.find_all('span', {'class': 'business-attribute price-range'})[0].string.encode('ascii','ignore')
    
    range_name = 'Unknown' if d.find_all('dd', {'class': 'nowrap price-description'})[0].string is None else\
    d.find_all('dd', {'class': 'nowrap price-description'})[0].string.encode('ascii','ignore').strip()
    
    address_street = 'Unknown' if d.find_all('span', {'itemprop': 'streetAddress'})[0].string is None else\
    d.find_all('span', {'itemprop': 'streetAddress'})[0].string.encode('ascii','ignore')
    
    address_city = 'Unknown' if d.find_all('span', {'itemprop': 'addressLocality'})[0].string is None else\
    d.find_all('span', {'itemprop': 'addressLocality'})[0].string.encode('ascii','ignore')
    
    address_state = 'Unknown' if d.find_all('span', {'itemprop': 'addressRegion'})[0].string is None else\
    d.find_all('span', {'itemprop': 'addressRegion'})[0].string.encode('ascii','ignore')
    
    address_zip = 'Unknown' if d.find_all('span', {'itemprop': 'postalCode'})[0].string is None else\
    d.find_all('span', {'itemprop': 'postalCode'})[0].string.encode('ascii','ignore')

    r_dict['restaurant_name'] = restaurant_name
    r_dict['review_rating'] = overall_rating
    r_dict['total_reviews'] = total_reviews
    r_dict['price_range'] = price_range
    r_dict['range_name'] = range_name
    r_dict['address_street'] = address_street
    r_dict['address_city'] = address_city
    r_dict['address_state'] = address_state
    r_dict['address_zip'] = address_zip

    
    return r_dict

In [3]:
def get_review_data(d):
    r_list = []
    rev_data1 = d.find_all("ul", { "class" : "user-passport-stats" })
    rev_data2 = d.find_all('li', {'class': 'user-location responsive-hidden-small'})
    rev_data3 = d.find_all('meta', {'itemprop': 'ratingValue'})[1:] #excludes first item which is overall rating
    rev_data4 = d.find_all('meta', {'itemprop': 'datePublished'})
    rev_data5 = d.find_all('p', {'itemprop': 'description'})
    
    for i in enumerate(rev_data1):
        r_dict = dict()
        r_dict['friend_count'] = '0' if (i[1].find_all('li', {'class': 'friend-count responsive-small-display-inline-block'})[0].find_all('b')[0].string) is None\
        else (i[1].find_all('li', {'class': 'friend-count responsive-small-display-inline-block'})[0].find_all('b')[0].string.encode('ascii','ignore'))  

        r_dict['review_count'] = '0' if (i[1].find_all('li', {'class': 'review-count responsive-small-display-inline-block'})[0].find_all('b')[0].string) is None\
        else (i[1].find_all('li', {'class': 'review-count responsive-small-display-inline-block'})[0].find_all('b')[0].string.encode('ascii','ignore'))

        r_dict['is_elite'] = 'No' if i[1].find_all('li', {'class': 'is-elite responsive-small-display-inline-block'}) == [] else 'Yes'

        rev_data2i = "Unknown" if rev_data2[i[0]].find_all('b')[0].string is None else rev_data2[i[0]].find_all('b')[0].string.encode('ascii', 'ignore')
        r_dict['reviewer_location'] = rev_data2i

        rev_data3i = 'Unknown' if rev_data3[i[0]].get('content') is None else rev_data3[i[0]].get('content')
        r_dict['review_rating'] = rev_data3i

        rev_data4i = 'Unknown' if rev_data4[i[0]].get('content') is None else rev_data4[i[0]].get('content')
        r_dict['review_date'] = rev_data4i

        rev_data5i = 'None' if rev_data5[i[0]].getText() is None else rev_data5[i[0]].getText().encode('ascii','ignore')
        r_dict['review_content'] = rev_data5i

        res_data5i = d.select('h1.biz-page-title.embossed-text-white')[0].string.encode('ascii','ignore').strip()
        r_dict['restaurant_name'] = res_data5i

        r_list.append(r_dict)
    
    return r_list


In [89]:
def url_cycle(u_file, p = 0):
    rest_list = []
    rev_list = []
    for i in u_file:
        try:
            f = bs4.BeautifulSoup(requests.get(i[0]).text)
            r = get_restaurant_data(f)
            total_review_count = int(r['total_reviews'])
            pages = range(0,total_review_count,20) # this is missing first page, url of 0 doesn't work
            print i
#             rest_list.append(r)
#             time.sleep(2)
        except Exception as e:
            print i
            print e
            print '_'*60
            pass
            
        for j in pages[p:len(pages)+1]:
            try:
                d = bs4.BeautifulSoup(requests.get(str(i[0])+'?sort_by=date_desc&start='+str(j)).text)
                rv = get_review_data(d)
#                 print rv
                print j
                rev_list.append(rv)
                time.sleep(4)
            except Exception as e2:
                print 'i = ' + str(i)
                print 'j = ' + str(j)
                print e2
                print '_'*60
                pass
            
    return rev_list #rest_list, rev_list               

In [90]:
michelin_rest_scrape = []
michelin_rev_scrape = []

In [91]:
michelin_rest_scrape.append(url_cycle(m_url[0:]))
michelin_rev_scrape.append(url_cycle(m_url[0:]))

['http://www.yelp.com/biz/solbar-calistoga']
700
720
740
760
['http://www.yelp.com/biz/luce-san-francisco']


In [26]:
pd.DataFrame(michelin_rest_scrape[0]).to_csv('non_michelin_restaurant_data.csv', header = True)

In [92]:
flattened_list = []
for x in michelin_rev_scrape:
    for y in x:
        flattened_list.append(y)
new_flattened_list = []
for x in flattened_list:
    for y in x:
        new_flattened_list.append(y)
# print new_flattened_list
pd.DataFrame(new_flattened_list).to_csv('michelin_review_data.csv', header = True)